# Histogram matching

This script takes in images and matches their pixel value histograms.

In [2]:
# some packages needed here
import cv2 as cv
import numpy as np
import glob
import os
from os import listdir
from os.path import isfile, join
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import match_histograms
import math

print("All libraries are loaded.")

All libraries are loaded.


## Input path

In [4]:
# give the input path for object images
parentPath = os.pardir
imagePath = glob.glob(parentPath + "/*/*/*/*.png")
imagePath.sort() # make sure the files are sorted

# count the number of images to process
nbrIm = len(imagePath)
print('There are', nbrIm, 'images to process.')

There are 583 images to process.


## Output path

In [5]:
# give the output path
outputPath = parentPath + r'/stimulus_set/'

# create the necessary output folder
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

## Histogram matching

### Obtaining the average image histogram

First, read all images and extract the average pixel value histogram from them. Note the next cell could take a while to execute.

In [6]:
# calculating the average histogram of all objects

# creating an empty vector of all possible pixel values
pixelValues = np.zeros([256])

# adding the number of pixels contained in every object to the vector
for file in imagePath:

    im = cv.imread(file, 0)

    for i in range(0,im.shape[0]):
        
        for j in range(0,im.shape[1]):
            
            pixelValues[im[i,j]] += 1

# creating the empty array that will hold all the value points of the histogram
ref_array = np.empty(int(np.sum(pixelValues)))

# fill in the empty array
idx = 0

pixels = []
for x in range(256): pixels.append(x)

for i in pixels:
    ref_array[idx : idx + int(pixelValues[i])] = i
    idx += int(pixelValues[i])

# transform the 1d array obtained into a readable image
ref_array = np.reshape(ref_array, (1, len(ref_array)))
ref_im = ref_array.astype('uint8')

### Equating image histograms

Next, read images, match their histogram to the average histogram, and write them. Note the next cell could take a while to execute.

In [7]:
for i in range(len(imagePath)):
    
    file = imagePath[i]
    name = os.path.splitext(os.path.basename(file))[0]
    
    im = cv.imread(file, 0)
    im = match_histograms(im, ref_im)
    
    cv.imwrite(outputPath + name + '.png', im)
    
    print(i, 'images processed.')